In [1]:
import numpy as np
import pandas as pd
# Set option manually
pd.set_option('display.max_columns', 40) 
pd.set_option('display.max_rows', 40)

# Compile Analysis Frame

Starting with outcomes DataFrame, it holds the "ground truth" of exciting projects from a business perspective. Applying cleaning:

In [36]:
dfout = pd.read_csv('Data/outcomes.csv', index_col=[0]) # ProjectID as index for easy merge

In [37]:
null_values_columns = dfout.columns[-3:] # from exploration notebook

# Cleaning 't' - 'f' values
dfout = dfout.assign(
                        **dfout.select_dtypes(include='object')
                            .replace({'f': 0, 't': 1})
                            .fillna(0)
                            .astype('uint'),

                        **dfout[null_values_columns]
                            .fillna(0)
                    )

dfout

,is_exciting,at_least_1_teacher_referred_donor,fully_funded,at_least_1_green_donation,great_chat,three_or_more_non_teacher_referred_donors,one_non_teacher_referred_donor_giving_100_plus,donation_from_thoughtful_donor,great_messages_proportion,teacher_referred_count,non_teacher_referred_count
projectid,,,,,,,,,,,
ffffc4f85b60efc5b52347df489d0238,0,0,0,0,0,0,0,0,0.0,0.0,0.0
ffffac55ee02a49d1abc87ba6fc61135,0,0,1,1,0,1,0,0,57.0,0.0,7.0
ffff97ed93720407d70a2787475932b0,0,0,1,1,1,1,1,0,100.0,0.0,3.0
ffff418bb42fad24347527ad96100f81,0,0,0,1,1,0,0,0,100.0,0.0,1.0
ffff2d9c769c8fb5335e949c615425eb,1,1,1,1,1,0,1,0,63.0,6.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...
0000ee613c92ddc5298bf63142996a5c,0,1,1,1,0,1,1,0,0.0,2.0,4.0
0000b38bbc7252972f7984848cf58098,0,0,1,1,0,0,1,0,50.0,0.0,2.0
00002d691c05c51a5fdfbb2baef0ba25,0,0,0,0,1,1,0,0,100.0,0.0,5.0


At this stage the outcomes DataFrame is cleaned.

Let's add information from the projects table. In the exploration notebook is noted how there are more projects in this new table than in the outcomes table, therefore, since we're using outcomes as the ground true (is_exciting column) let's enrich only for the projects in the outcomes table.

Let's import 'projects.csv', for the relevant columns (from exploration) and do basic cleaning

In [56]:
# Text 
text_relv_col = [
    'poverty_level', 'resource_type','primary_focus_subject', 'primary_focus_area', 'school_metro'
    ]

bin_relv_col = [
    'school_year_round', 'school_charter', 'school_magnet', 'eligible_double_your_impact_match',
    'eligible_almost_home_match'
]

cont_relv_col = [
    'fulfillment_labor_materials', 'total_price_excluding_optional_support', 
    'total_price_including_optional_support', 'students_reached'
]

date_col = ['date_posted']

In [57]:
dfprj = pd.read_csv('Data/projects.csv', index_col=[0], usecols=(['projectid'] + text_relv_col + bin_relv_col +cont_relv_col + date_col) )
dfprj

,school_metro,school_charter,school_magnet,school_year_round,primary_focus_subject,primary_focus_area,resource_type,poverty_level,fulfillment_labor_materials,total_price_excluding_optional_support,total_price_including_optional_support,students_reached,eligible_double_your_impact_match,eligible_almost_home_match,date_posted
projectid,,,,,,,,,,,,,,,
316ed8fb3b81402ff6ac8f721bb31192,NaN,f,f,f,Literature & Writing,Literacy & Language,Books,highest poverty,30.0,555.81,653.89,32.0,f,f,2014-05-12
90de744e368a7e4883223ca49318ae30,urban,f,f,f,Literacy,Literacy & Language,Books,highest poverty,30.0,296.47,348.79,22.0,f,f,2014-05-12
32943bb1063267de6ed19fc0ceb4b9a7,rural,f,f,f,Literacy,Literacy & Language,Technology,high poverty,30.0,430.89,506.93,17.0,f,f,2014-05-11
bb18f409abda2f264d5acda8cab577a9,urban,f,t,f,Social Sciences,History & Civics,Books,highest poverty,30.0,576.07,677.73,12.0,f,f,2014-05-11
24761b686e18e5eace634607acbcc19f,urban,f,f,f,Mathematics,Math & Science,Other,highest poverty,30.0,408.40,480.47,24.0,f,f,2014-05-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
a7236ea96c812895cafc5d700d779147,urban,f,f,f,Environmental Science,Math & Science,Supplies,highest poverty,NaN,231.00,281.71,0.0,f,f,2002-09-17
e02da37beb332eb66c2d2ba989c597ad,urban,f,f,f,Economics,History & Civics,Technology,highest poverty,NaN,1129.00,1376.83,0.0,f,f,2002-09-17
82e536f14eadf2671a70e03416f695a3,urban,f,t,f,Early Development,Applied Learning,Supplies,moderate poverty,NaN,125.00,152.44,0.0,f,f,2002-09-16


---- quick refresh ----

In [58]:
dfprj[text_relv_col].isna().sum()
# fill nan in school_metro with a new type for the empties, is the same quantity than rural.

poverty_level                0
resource_type               45
primary_focus_subject       39
primary_focus_area          39
school_metro             81908
dtype: int64

In [59]:
dfprj[bin_relv_col].isna().sum()

school_year_round                    0
school_charter                       0
school_magnet                        0
eligible_double_your_impact_match    0
eligible_almost_home_match           0
dtype: int64

In [62]:
dfprj[cont_relv_col].isna().sum() # Fill with 0

fulfillment_labor_materials               35082
total_price_excluding_optional_support        0
total_price_including_optional_support        0
students_reached                            146
dtype: int64